# Chapter 2: Data Ingestion & Variables - Exercise

In [ ]:
low_memory=False
import numpy as np
import pandas as pd

Once deleted, variables cannot be recovered. Proceed (y/[n])?  y


## 2.1 Introduction and Problem Setting

Today you begin work on a practical fraud detection exercise.

A neighbouring company has reported a new type of credit card fraud. The exact cause is unknown but the fraudster can create multiple transactions at the same time, which may result in the same amount being recorded more than once. Your task is to check whether our company is affected and, if so, identify any client linked to suspicious transactions.

You receive three datasets from our offices:

- London (all transactions in UTC time): `Transactions_UTC_upload.txt`
- Chicago (US Central time): `Transactions_US_Central_upload.txt`
- Shanghai: `Transactions_China_upload.txt`

Client details are in `Credit_customers.txt`.

If you find suspicious activity, report the client name(s) associated with those transactions.

## 2.2 Data ingestion and exploration

First, load each dataset into its own `pandas` DataFrame. Inspect the first rows, the column names, and the column types to get a quick overview.

Notes about `pd.read_csv()` options you will see in the code below:

- `sep` sets the field delimiter used in the file. Common values are `','` for comma separated files and `';'` or `'\t'` for semicolon or tab separated files. Use the correct `sep` so `pandas` splits columns properly.
- `index_col=0` tells `pandas` to use the first column in the file as the DataFrame index (row labels) rather than the default numeric row numbers. This is useful when the file already contains an ID or name column you want to use to reference rows.

Example: `pd.read_csv('Credit_customers.txt', sep='\t', index_col=0)` reads a tab separated file and uses the first column as the row index.

In [4]:
Credit_clients = pd.read_csv("Credit_customers.txt", sep='\t', index_col = 0)
Credit_clients.head()

,Name
Client_ID,
1,"Braund, Mr. Owen Harris"
2,"Cumings, Mrs. John Bradley (Florence Briggs Th..."
3,"Heikkinen, Miss. Laina"
4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)"
5,"Allen, Mr. William Henry"


In [7]:
Credit_clients.dtypes

Name    object
dtype: object

In [10]:
US = pd.read_csv("Transactions_US_Central_upload.txt", sep = ";")
US.head()

,Client_ID,Transaction_time,time_zone
0,148,2019-12-15 05:24:35,US_Central
1,299,2019-12-15 13:09:57,US_Central
2,197,2019-12-15 13:30:02,US_Central
3,259,2019-12-15 14:26:03,US_Central
4,591,2019-12-16 06:31:55,US_Central


In [12]:
US.dtypes

Client_ID            int64
Transaction_time    object
time_zone           object
dtype: object

In [11]:
EU = pd.read_csv("Transactions_UTC_upload.txt", sep = ";")
EU.head()

,Client_ID,Transaction_time,time_zone
0,86,2019-12-15 13:39:21,UTC
1,313,2019-12-16 18:00:26,UTC
2,198,2019-12-17 06:20:07,UTC
3,466,2019-12-17 06:47:42,UTC
4,119,2019-12-17 11:46:27,UTC


In [13]:
EU.dtypes

Client_ID            int64
Transaction_time    object
time_zone           object
dtype: object

In [9]:
China = pd.read_csv("Transactions_China_upload.txt", sep = ";")
China.head()

,Client_ID,Transaction_time,time_zone
0,805,2019-12-15 03:37:36,China
1,67,2019-12-15 06:42:35,China
2,749,2019-12-15 14:48:02,China
3,828,2019-12-15 22:17:06,China
4,539,2019-12-15 23:30:35,China


In [6]:
China.dtypes

Client_ID            int64
Transaction_time    object
time_zone           object
dtype: object

## 2.3 Data Processing

After loading the files, convert the transaction timestamp columns to `datetime` objects so you can compare times across sources. When data comes from different time zones, localize each timestamp to its original zone and then convert to a common timezone such as UTC before comparing.

Now look for duplicate timestamps that may indicate multiple simultaneous transactions. If you are unsure how to proceed, search for common techniques to compare datasets or to find duplicate timestamps. Try to reason about timezone differences and whether identical UTC timestamps could indicate suspicious behavior.

In [14]:
EU['Transaction_time'] = pd.to_datetime(EU['Transaction_time'], utc = True) 
US['Transaction_time'] = pd.to_datetime(US['Transaction_time']) 
China['Transaction_time'] = pd.to_datetime(China['Transaction_time']) 

In [15]:
China['Transaction_time'] = China['Transaction_time'].dt.tz_localize(tz = "Asia/Shanghai").dt.tz_convert('UTC')
US['Transaction_time'] = US['Transaction_time'].dt.tz_localize(tz = "US/central").dt.tz_convert('UTC')

In [27]:
transactions = pd.concat([EU, US, China], axis=0)

In [29]:
transactions.head()

,Client_ID,Transaction_time,time_zone
0,86,2019-12-15 13:39:21+00:00,UTC
1,313,2019-12-16 18:00:26+00:00,UTC
2,198,2019-12-17 06:20:07+00:00,UTC
3,466,2019-12-17 06:47:42+00:00,UTC
4,119,2019-12-17 11:46:27+00:00,UTC


In [32]:
duplicates = transactions['Transaction_time'].duplicated(keep=False)

In [33]:
transactions[duplicates]

,Client_ID,Transaction_time,time_zone
319,577,2020-03-02 04:59:43+00:00,UTC
307,577,2020-03-02 04:59:43+00:00,US_Central
336,577,2020-03-02 04:59:43+00:00,China


If you find duplicate transaction timestamps, try to link those rows to the client table using the customer ID to see whether a known client appears in the suspicious records.

In [34]:
Credit_clients.loc[577]

Name    Nigerian Prince
Name: 577, dtype: object